# Modeling data

Lets select the best model among logistic regression, svm, random forest, gbm & neural network.

Will be using 5 fold cross validation in the model training process and Grid search for the model selection.

In [2]:
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score

# from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

#### Load feature engineered train, test datasets

In [3]:
x_train = pd.read_csv('data/train_features.csv')
x_test = pd.read_csv('data/test_features.csv')

y_train = pd.read_csv('data/train_labels.csv')
y_test = pd.read_csv('data/test_labels.csv')

#### For model evaluation

In [4]:
model_acc = {}
model_f1 = {}

#### Functions to evaluate cross validation results

In [5]:
# Function to evaluate cross validation results
def print_results(results, model):

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('Acc: {} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))
        
    print('\nBest Params: {},'.format(results.best_params_), 'Accuracy: {}'.format(results.best_score_))
    # add to model_acc dict
    model_acc[model] = str(round(results.best_score_,3))
        
# Function to get f1 score of the best model
def best_model_f1_score(results, model):
    y_pred = results.best_estimator_.predict(x_test)
    
    f1 = f1_score(y_test, y_pred)
    # add to model_f1 dict
    model_f1[model] = round(f1,3)

    return f1

### Logistic Regression

In [6]:
lr = LogisticRegression()
parameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

lr_cv = GridSearchCV(lr, parameters, cv=5)
lr_cv.fit(x_train, y_train.values.ravel())

# Find best model
print_results(lr_cv,'lr')

Acc: 0.717 (+/-0.12) for {'C': 0.001}
Acc: 0.731 (+/-0.11) for {'C': 0.01}
Acc: 0.748 (+/-0.106) for {'C': 0.1}
Acc: 0.752 (+/-0.097) for {'C': 1}
Acc: 0.752 (+/-0.096) for {'C': 10}
Acc: 0.752 (+/-0.097) for {'C': 100}
Acc: 0.752 (+/-0.097) for {'C': 1000}

Best Params: {'C': 10}, Accuracy: 0.752422480620155


#### F1 score gives an idea about both precision and recall. It is a better metric when there are imbalanced classes.

In [7]:
print("f1 score - lr:", best_model_f1_score(lr_cv,'lr'))

f1 score - lr: 0.45484581497797355


#### Write out pickled model

In [8]:
# Write out pickled model
joblib.dump(lr_cv.best_estimator_, 'models/lr_model.pkl')

['models/lr_model.pkl']

- Logistic Regression model with the best hyperparameter combination has accuracy of 0.752 and a f1-score of 0.454.
##### Let's evaluate other models as well.

### Support Vector Machine

In [9]:
svc = SVC()
parameters = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10]
}

svm_cv = GridSearchCV(svc, parameters, cv=5)
svm_cv.fit(x_train, y_train.values.ravel())

print_results(svm_cv,'svm')

Acc: 0.749 (+/-0.103) for {'C': 0.1, 'kernel': 'linear'}
Acc: 0.77 (+/-0.124) for {'C': 0.1, 'kernel': 'rbf'}
Acc: 0.751 (+/-0.098) for {'C': 1, 'kernel': 'linear'}
Acc: 0.807 (+/-0.1) for {'C': 1, 'kernel': 'rbf'}
Acc: 0.752 (+/-0.098) for {'C': 10, 'kernel': 'linear'}
Acc: 0.822 (+/-0.086) for {'C': 10, 'kernel': 'rbf'}

Best Params: {'C': 10, 'kernel': 'rbf'}, Accuracy: 0.8220930232558139


In [10]:
print("f1 score - svm:", best_model_f1_score(svm_cv,'svm'))

f1 score - svm: 0.5703794369645043


In [11]:
# Write out pickled model
joblib.dump(svm_cv.best_estimator_, 'models/svm_model.pkl')

['models/svm_model.pkl']

### Random Forest

In [12]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5, 50, 250],
    'max_depth': [2, 4, 8, 16, 32, None]
}

rf_cv = GridSearchCV(rf, parameters, cv=5)
rf_cv.fit(x_train, y_train.values.ravel())

print_results(rf_cv,'rf')

Acc: 0.73 (+/-0.106) for {'max_depth': 2, 'n_estimators': 5}
Acc: 0.764 (+/-0.097) for {'max_depth': 2, 'n_estimators': 50}
Acc: 0.768 (+/-0.086) for {'max_depth': 2, 'n_estimators': 250}
Acc: 0.772 (+/-0.085) for {'max_depth': 4, 'n_estimators': 5}
Acc: 0.794 (+/-0.09) for {'max_depth': 4, 'n_estimators': 50}
Acc: 0.8 (+/-0.094) for {'max_depth': 4, 'n_estimators': 250}
Acc: 0.812 (+/-0.073) for {'max_depth': 8, 'n_estimators': 5}
Acc: 0.824 (+/-0.073) for {'max_depth': 8, 'n_estimators': 50}
Acc: 0.825 (+/-0.076) for {'max_depth': 8, 'n_estimators': 250}
Acc: 0.816 (+/-0.083) for {'max_depth': 16, 'n_estimators': 5}
Acc: 0.848 (+/-0.077) for {'max_depth': 16, 'n_estimators': 50}
Acc: 0.852 (+/-0.077) for {'max_depth': 16, 'n_estimators': 250}
Acc: 0.821 (+/-0.066) for {'max_depth': 32, 'n_estimators': 5}
Acc: 0.845 (+/-0.082) for {'max_depth': 32, 'n_estimators': 50}
Acc: 0.85 (+/-0.077) for {'max_depth': 32, 'n_estimators': 250}
Acc: 0.818 (+/-0.071) for {'max_depth': None, 'n_estim

In [13]:
print("f1 score - rf:", best_model_f1_score(rf_cv,'rf'))

f1 score - rf: 0.5905256327060351


Note: The default value for max_depth is None, which means that each tree will expand until every leaf is pure. A pure leaf is one where all of the data on the leaf comes from the same class.

In [14]:
# Write out pickled model
joblib.dump(rf_cv.best_estimator_, 'models/rf_model.pkl')

['models/rf_model.pkl']

### Gradient Boosting Model

In [15]:
gb = GradientBoostingClassifier()
parameters = {
    'n_estimators': [5, 50, 250, 500],
    'max_depth': [1, 3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 1]
}

gbm_cv = GridSearchCV(gb, parameters, cv=5)
gbm_cv.fit(x_train, y_train.values.ravel())

print_results(gbm_cv,'gbm')

Acc: 0.68 (+/-0.094) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 5}
Acc: 0.705 (+/-0.04) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 50}
Acc: 0.748 (+/-0.076) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 250}
Acc: 0.775 (+/-0.077) for {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 500}
Acc: 0.748 (+/-0.087) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 5}
Acc: 0.754 (+/-0.087) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
Acc: 0.807 (+/-0.061) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 250}
Acc: 0.819 (+/-0.064) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
Acc: 0.777 (+/-0.075) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 5}
Acc: 0.79 (+/-0.061) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
Acc: 0.821 (+/-0.069) for {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 250}
Acc: 0.831 (+/-0.071) for {'learning_rate': 0.01, 'max_

In [16]:
print("f1 score - gbm:", best_model_f1_score(gbm_cv,'gbm'))

f1 score - gbm: 0.5643496214728148


In [17]:
# Write out pickled model
joblib.dump(gbm_cv.best_estimator_, 'models/gbm_model.pkl')

['models/gbm_model.pkl']

### Artificial Neural Network

In [18]:
ann_model = Sequential()

ann_model.add(Dense(1024, input_dim = 10, activation = 'relu'))
ann_model.add(Dropout(0.4))

ann_model.add(Dense(256, activation = 'relu'))
ann_model.add(Dropout(0.4))

ann_model.add(Dense(64, activation = 'relu'))
ann_model.add(Dropout(0.4))

ann_model.add(Dense(32, activation = 'relu'))

ann_model.add(Dense(1, activation = 'sigmoid'))

ann_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              11264     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2

In [19]:
# Compile model
ann_model.compile(loss='mse', optimizer=Adam(), metrics=['accuracy'])

In [20]:
# train model
history = ann_model.fit(x_train, y_train, epochs=25, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/25
323/323 [==============================] - 3s 9ms/step - loss: 0.1799 - accuracy: 0.7321 - val_loss: 0.1951 - val_accuracy: 0.7157
Epoch 2/25
323/323 [==============================] - 3s 8ms/step - loss: 0.1594 - accuracy: 0.7679 - val_loss: 0.1868 - val_accuracy: 0.7257
Epoch 3/25
323/323 [==============================] - 2s 7ms/step - loss: 0.1475 - accuracy: 0.7878 - val_loss: 0.1371 - val_accuracy: 0.8057
Epoch 4/25
323/323 [==============================] - 3s 8ms/step - loss: 0.1438 - accuracy: 0.7952 - val_loss: 0.1533 - val_accuracy: 0.7846
Epoch 5/25
323/323 [==============================] - 3s 8ms/step - loss: 0.1393 - accuracy: 0.7984 - val_loss: 0.1345 - val_accuracy: 0.8174
Epoch 6/25
323/323 [==============================] - 3s 10ms/step - loss: 0.1377 - accuracy: 0.8029 - val_loss: 0.1548 - val_accuracy: 0.7863
Epoch 7/25
323/323 [==============================] - 3s 9ms/step - loss: 0.1343 - accuracy: 0.8104 - val_loss: 0.1658 - val_accuracy: 0.7634
Epoch

#### Evaluate ANN model

In [22]:
# evaluate the model
_, train_acc = ann_model.evaluate(x_train, y_train, verbose=0)
_, test_acc = ann_model.evaluate(x_test, y_test, verbose=0)
print('Train acc: %.4f, Test acc: %.4f' % (train_acc, test_acc))

# add to model_acc dict
model_acc['ann'] = round(test_acc,3)

ann_y_preds = ann_model.predict_classes(x_test)

# Get f1 score
f1_ann = f1_score(y_test, ann_y_preds)
print("ann:", f1_ann)

# add to model_f1 dict
model_f1['ann'] = round(f1_ann,3)

Train acc: 0.8437, Test acc: 0.7937
ann: 0.5732860520094563


In [23]:
# Write out keras model
ann_model.save('models/ann_model.h5')

## Evaluation of models

In [36]:
acc_f1_list = [model_acc, model_f1]

df_eval = pd.DataFrame.from_records(acc_f1_list).T
df_eval.rename(columns = {0:'Acc', 1:'F1'}, inplace = True)

display(df_eval)

,Acc,F1
lr,0.752,0.455
svm,0.822,0.57
rf,0.852,0.591
gbm,0.872,0.564
ann,0.794,0.573


 - Low F1 scores are seen in all models due to the imbalanced nature of the dataset.
 - Evaluation of Accuracy and F1 scores among all models suggests that Support Vector Machine, Random Forest & Gradient Boosted Models are the best performers.
 - Although GBM has the best accuracy it's low f1 score compared to others gives the edge to the ***Random Forest Model***.